<h1>삼성전자(005930) 연결 재무상태표 DB에 넣기</h1>

In [1]:
import requests
from io import BytesIO
import pandas as pd
import numpy as np
import re
import os
import mysql.connector
from sqlalchemy import create_engine
from datetime import datetime

In [2]:
# opendart API로 상장사 사업보고서의 rcp_no 불러오기 
api_key = '27d86452b033fe973df6f1c980c37a2586801a1a'
corp_code = '00126380' # 삼성전자 corp_code
bgn_de = datetime.today().replace(year=2017).strftime('%Y%m%d')
end_de = datetime.today().strftime('%Y%m%d')

url = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de={}&end_de={}&last_reprt_at=Y&pblntf_detail_ty=A001&page_no=1&page_count=10'.format(api_key, corp_code, bgn_de, end_de)

In [3]:
resp = requests.get(url)
webpage = resp.content.decode('UTF-8')
webpage

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><result><status>000</status><message>정상</message><page_no>1</page_no><page_count>10</page_count><total_count>3</total_count><total_page>1</total_page><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2018.12)</report_nm><rcept_no>20190401004781</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20190401</rcept_dt><rm>연</rm></list><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2017.12)</report_nm><rcept_no>20180402005019</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20180402</rcept_dt><rm>연</rm></list><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2016.12)</report_nm><rcept_no>20170331004518</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20170331</rcept_dt><rm>연</rm></list></result>'

In [4]:
stock_code = re.findall(r'<stock_code>(.*?)</stock_code>', webpage)[0]
rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', webpage)
period_list = re.findall(r'<report_nm>사업보고서 \((.*?)\)</report_nm>', webpage)

In [7]:
stock_code

'005930'

In [8]:
period_list

['2018.12', '2017.12', '2016.12']

In [9]:
# 각 rcp_no의 pair number인 dcm_no 조회
dcm_no_list = []
for rcp_no in rcp_no_list:
    resp = requests.get('http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no))
    webpage = resp.content.decode('utf-8')
    dcm_no = re.findall(r"'{}', '(.*?)',".format(rcp_no), webpage)[0]
    dcm_no_list.append(dcm_no)

In [10]:
dcm_no_list

['6616741', '6060273', '5540271']

In [11]:
# 파악된 rcp_no와 dcm_no 정보를 url에 넣어 재무제표 excel 파일을 pandas로 불러오기
user_agent = "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"

url = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko".format(rcp_no_list[0],dcm_no_list[0])
resp = requests.get(url, headers={"user-agent": user_agent})
table = BytesIO(resp.content)

# 재무제표별 DataFrame 생성
df_bs_con = pd.read_excel(table, sheet_name="연결 재무상태표", skiprows=6)
df_bs_sep = pd.read_excel(table, sheet_name="재무상태표", skiprows=6)
df_is_con = pd.read_excel(table, sheet_name="연결 손익계산서", skiprows=6)
df_is_sep = pd.read_excel(table, sheet_name="손익계산서", skiprows=6)
df_cf_con = pd.read_excel(table, sheet_name="연결 현금흐름표", skiprows=6)
df_cf_sep = pd.read_excel(table, sheet_name="현금흐름표", skiprows=6)


In [12]:
period_list.insert(0, 'account')

In [13]:
period_list

['account', '2018.12', '2017.12', '2016.12']

In [14]:
# 각 df별 column 설정
df_bs_con.columns = period_list
df_bs_sep.columns = period_list
df_is_con.columns = period_list
df_is_sep.columns = period_list 
df_cf_con.columns = period_list
df_cf_sep.columns = period_list

In [15]:
df_cf_sep

,account,2018.12,2017.12,2016.12
0,영업활동 현금흐름,44341217,38906190,23984804
1,영업에서 창출된 현금흐름,53596311,41350471,24901464
2,당기순이익,32815127,28800837,11579749
3,조정,27095149,18012976,14910093
4,영업활동으로 인한 자산부채의 변동,-6313965,-5463342,-1588378
5,이자의 수취,459074,491501,622118
6,이자의 지급,-343270,-265364,-208010
7,배당금 수입,779567,1118779,903758
8,법인세 납부액,-10150465,-3789197,-2234526
9,투자활동 현금흐름,-31678548,-28118806,-14240450


In [16]:
# 계정명 내 공백제거
df_bs_con['account'] = df_bs_con['account'].str.strip()
df_bs_sep['account'] = df_bs_sep['account'].str.strip()
df_is_con['account'] = df_is_con['account'].str.strip()
df_is_sep['account'] = df_is_sep['account'].str.strip()
df_cf_con['account'] = df_cf_con['account'].str.strip()
df_cf_sep['account'] = df_cf_sep['account'].str.strip()

In [17]:
df_is_con

,account,2018.12,2017.12,2016.12
0,수익(매출액),243771415,239575376,201866745
1,매출원가,132394411,129290661,120277715
2,매출총이익,111377004,110284715,81589030
3,판매비와관리비,52490335,56639677,52348358
4,영업이익(손실),58886669,53645038,29240672
5,기타수익,1485037,3010657,3238261
6,기타비용,1142018,1419648,2463814
7,지분법이익,539845,201442,19501
8,금융수익,9999321,9737391,11385645
9,금융비용,8608896,8978913,10706613


In [18]:
# 공백란을 0로 채우기
df_bs_con = df_bs_con.replace(" ", 0)
df_bs_sep = df_bs_sep.replace(" ", 0)
df_is_con = df_is_con.replace(" ", 0)
df_is_sep = df_is_sep.replace(" ", 0)
df_cf_con = df_cf_con.replace(" ", 0)
df_cf_sep = df_cf_sep.replace(" ", 0)

In [19]:
# input_dict = "'{}':'str','{}':'int','{}':'int','{}':'int'".format(period_list[0], period_list[1], period_list[2], period_list[3])

df_bs_con.astype({period_list[0]:'str',
                  period_list[1]:'int',
                  period_list[2]:'int',
                  period_list[3]:'int'})
# df_bs_sep.astype({input_dict})
# df_is_con.astype({input_dict})
# df_is_sep.astype({input_dict})
# df_cf_con.astype({input_dict})
# df_cf_sep.astype({input_dict})

,account,2018.12,2017.12,2016.12
0,자산,0,0,0
1,유동자산,174697424,146982464,141429704
2,현금및현금성자산,30340505,30545130,32111442
3,단기금융상품,65893797,49447696,52432411
4,단기매도가능금융자산,0,3191375,3638460
5,단기상각후원가금융자산,2703693,0,0
6,단기당기손익-공정가치금융자산,2001948,0,0
7,매출채권,33867733,27695995,24279211
8,미수금,3080733,4108961,3521197
9,선급금,1361807,1753673,1439938


In [20]:
df_cf_sep.dtypes

account    object
2018.12     int64
2017.12     int64
2016.12     int64
dtype: object

In [21]:
# df_bs_con, df_bs_sep의 자산, 부채, 자본 항목 인덱싱
ale_tuple = ('자산', '부채', '자본')

bs_beg_idx = []
bs_end_idx = []

for val in ale_tuple:
    bs_beg_idx.append(df_bs_con.loc[df_bs_con['account'] == val,:].index[0] + 1)
    bs_end_idx.append(df_bs_con.loc[df_bs_con['account'] == val+'총계',:].index[0])
bs_end_idx

[27, 49, 60]

In [22]:
# df_cf_con, df_cf_sep의 영업/투자/재무 활동 현금흐름 항목 인덱싱
oif_tuple = ('영업활동 현금흐름', '투자활동 현금흐름', '재무활동 현금흐름')

cf_beg_idx = []

for val in oif_tuple:
    cf_beg_idx.append(df_cf_con.loc[df_cf_con['account'] == val,:].index[0])

cf_beg_idx

[0, 9, 34]

In [23]:
bs_beg_idx

[1, 29, 51]

In [24]:
bs_end_idx

[27, 49, 60]

In [78]:
df_bs_con.loc[i,df_bs_con.columns[0]]

'자본과부채총계'

In [107]:
#connect MySQL
mydb = mysql.connector.connect(host = 'localhost',
                                user = 'root',
                                passwd = 'root',
                                db = 'fsdb')
mycursor = mydb.cursor()

In [105]:
# BS table 만들기(consolidated)
def create_bs_con_table(stock_code, df_bs_con, bs_beg_idx, bs_end_idx):
    ale_eng_tuple = ('asset', 'liability', 'equity')

    for beg, end, ale in zip(bs_beg_idx, bs_end_idx, ale_eng_tuple):
        print("beg={}, end={}, ale={}".format(beg, end, ale))
        # 자산, 부채, 자본항목별로 테이블 자르기
        table = df_bs_con.loc[beg:end,:]

        # DB에 자산, 부채, 자본항목별 테이블 생성
        sql = """CREATE TABLE {}_bs_con_{} (
                        id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                        account varchar(255),
                        FY18 int(20),
                        FY17 int(20),
                        FY16 int(20),
                        PRIMARY KEY (id));""".format(stock_code, ale)
        
        mycursor.execute(sql)

        # 각 테이블에 관련 계정명과 수치 입력
        for i in table.index:
            sql2 = """INSERT INTO {}_bs_con_{} (account, FY18, FY17, FY16) 
                    VALUES ("{}", {}, {}, {})""".format(stock_code, ale, table.loc[i,table.columns[0]], table.loc[i,table.columns[1]], table.loc[i,table.columns[2]], table.loc[i,table.columns[3]])
            mycursor.execute(sql2)
        mydb.commit()
    
    return "bs_con 업데이트 성공"

In [108]:
# BS table 만들기(separated)
def create_bs_sep_table(stock_code, df_bs_sep, bs_beg_idx, bs_end_idx):
    ale_eng_tuple = ('asset', 'liability', 'equity')

    for beg, end, ale in zip(bs_beg_idx, bs_end_idx, ale_eng_tuple):
        print("beg={}, end={}, ale={}".format(beg, end, ale))
        # 자산, 부채, 자본항목별로 테이블 자르기
        table = df_bs_sep.loc[beg:end,:]

        # DB에 자산, 부채, 자본항목별 테이블 생성
        sql = """CREATE TABLE {}_bs_sep_{} (
                        id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                        account varchar(255),
                        FY18 int(20),
                        FY17 int(20),
                        FY16 int(20),
                        PRIMARY KEY (id));""".format(stock_code, ale)
        
        mycursor.execute(sql)

        # 각 테이블에 관련 계정명과 수치 입력
        for i in table.index:
            sql2 = """INSERT INTO {}_bs_sep_{} (account, FY18, FY17, FY16) 
                    VALUES ("{}", {}, {}, {})""".format(stock_code, ale, table.loc[i,table.columns[0]], table.loc[i,table.columns[1]], table.loc[i,table.columns[2]], table.loc[i,table.columns[3]])
            mycursor.execute(sql2)
        mydb.commit()
    
    return "bs_sep 업데이트 성공"

In [106]:
create_bs_con_table(stock_code, df_bs_con, bs_beg_idx, bs_end_idx)

beg=1, end=27, ale=asset
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
beg=29, end=49, ale=liability
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
beg=51, end=60, ale=equity
51
52
53
54
55
56
57
58
59
60


'bs_con 업데이트 성공'

In [109]:
create_bs_sep_table(stock_code, df_bs_sep, bs_beg_idx, bs_end_idx)

beg=1, end=27, ale=asset
beg=29, end=49, ale=liability
beg=51, end=60, ale=equity


'bs_sep 업데이트 성공'

In [34]:
# 입력 확인
sql = "SELECT * FROM {}_bs_con_{}".format(stock_code, ale_eng_tuple[2])
pd.read_sql_query(sql, mydb)

,id,account,FY18,FY17,FY16
0,1,유동부채,69081510,67175114,54704095
1,2,매입채무,8479916,9083907,6485039
2,3,단기차입금,13586660,15767619,12746789
3,4,미지급금,10711536,13899633,11525910
4,5,선수금,820265,1249174,1358878
5,6,예수금,951254,793582,685028
6,7,미지급비용,20339687,13996273,12527300
7,8,미지급법인세,8720050,7408348,2837353
8,9,유동성장기부채,33386,278619,1232817
9,10,충당부채,4384038,4294820,4597417


In [103]:
mydb.close()